# edamam API
- [docs](https://developer.edamam.com/edamam-docs-recipe-api)


In [1]:
import json
import requests

import pandas as pd

In [2]:
# fetch api key from secrets.json file
with open("secrets.json", "rb") as f:
    key_file = json.loads(f.read())

app_id = key_file["app_id"]
app_key = key_file["key"]

- [sample reponse](/sample_response.json) from edamam "get recipes" endpoint with "q=cherry tomatoes, sourdough bread" and "ingr=3-7"

In [3]:
def get_recipes(ingr: list[str], num_ingr: tuple[int, int], dish: str, meal: str, app_id: str, app_key: str):
    recipes_endpoint = "https://api.edamam.com/api/recipes/v2?type=public"
    ingredients = f"q={'%2C%20'.join(ingr)}"  # TODO: handle empty `ingr` list
    min_max_ingr = f"ingr={min(num_ingr)}-{max(num_ingr)}"  # TODO: handle empty or single-element `num_ingr`
    dish_type=f"dishType={dish}"  # TODO: set default `dish`?
    meal_type = f"mealType={meal}"  # TODO: set default `meal`?
    # full_url = f"{recipes_endpoint}&{ingredients}&{min_max_ingr}&{dish_type}&{meal_type}&app_id={app_id}&app_key={app_key}"
    full_url = f"{recipes_endpoint}&{ingredients}&{min_max_ingr}&{dish_type}&app_id={app_id}&app_key={app_key}"
    print(full_url)
    response = requests.get(full_url)
    data = response.json()
    print(response.status_code)
    
    if response.status_code == 200 and data["count"] > 0:
        items = []
        for item in data["hits"]:
            recipe_dict = {}
            recipe_dict["title"] = item["recipe"]["label"]
            recipe_dict["link"] = item["recipe"]["url"]
            recipe_dict["servings"] = item["recipe"]["yield"]
            recipe_dict["cuisine"] = item["recipe"]["cuisineType"]
            recipe_dict["diet_labels"] = item["recipe"]["dietLabels"]
            recipe_dict["healh_labels"] = item["recipe"]["healthLabels"]
            recipe_dict["cautions"] = item["recipe"]["cautions"]
            recipe_dict["ingredient_lines"] = item["recipe"]["ingredientLines"]
            
            ingreds = []
            for ingr in item["recipe"]["ingredients"]:
                ingredient_details = {}
                ingredient_details["text"] = ingr["text"]
                ingredient_details["quantity"] = ingr["quantity"]
                ingredient_details["measure"] = ingr["measure"]
                ingredient_details["weight"] = ingr["weight"]
                ingredient_details["food"] = ingr["food"]
                ingredient_details["food_category"] = ingr["foodCategory"]
                ingreds.append(ingredient_details)
                recipe_dict["ingredient_details"] = ingreds
            items.append(recipe_dict)
        
    return pd.DataFrame.from_dict(items)

    

| dishType options | mealType options |
| --- | --- |
| Biscuits and cookies, Bread, Cereal, Condiments and sauces, Desserts, Drinks,  Main course, Pancake, Preps, Preserve, Salad, Sandwiches, Soup, Starter, Sweets | Breakfast, Dinner, Lunch, Snack, Teatime |

In [4]:
test_ingr = ["cherry%20tomatoes", "sourdough%20bread"]
test_min_max = (3,7)
test_dish = "Main%20course"
test_meal = "Breakfast"

In [5]:
df = get_recipes(ingr=test_ingr, num_ingr=test_min_max, dish=test_dish, meal=test_meal, app_id=app_id, app_key=app_key)

https://api.edamam.com/api/recipes/v2?type=public&q=cherry%20tomatoes%2C%20sourdough%20bread&ingr=3-7&dishType=Main%20course&app_id=ebe7f48e&app_key=2b1886dad7aff33d1d0cba45c943fd60
200


In [6]:
df.columns

Index(['title', 'link', 'servings', 'cuisine', 'diet_labels', 'healh_labels',
       'cautions', 'ingredient_lines', 'ingredient_details'],
      dtype='object')

In [7]:
df["ingredient_details"].head()

0    [{'text': '* 8 to 12 cherry tomato, depending ...
1    [{'text': 'One large slice of wholemeal or sou...
2    [{'text': '2 cherry tomatoes or 1 roma tomato,...
3    [{'text': '6 eggs', 'quantity': 6.0, 'measure'...
4    [{'text': '5 cherry tomatoes', 'quantity': 5.0...
Name: ingredient_details, dtype: object

----
## Exploring FooDB data
Downloaded the "FooDB CSV File" from [here](https://foodb.ca/downloads), which turned out to be a zip file with about 2 dozen csv files, some of them bigger than git can handle. I put them all in the "data/foodb_2020_04_07_csv" folder & then added that to my gitignore.

If I end up using any of the ones that are too big, I'll have to figure out how to make them available to my app. Lots of possibilities for that.

In [8]:
flavor = pd.read_csv("data/foodb_2020_04_07_csv/Flavor.csv")
flavor.head()

,id,name,flavor_group,category,created_at,updated_at,creator_id,updater_id
0,1,celery,vegetable,odor,2011-10-02 06:10:04 UTC,2011-10-02 06:10:04 UTC,NaN,NaN
1,2,corn,vegetable,odor,2011-10-02 06:10:04 UTC,2011-10-02 06:10:04 UTC,NaN,NaN
2,3,cucumber,vegetable,odor,2011-10-02 06:10:04 UTC,2011-10-02 06:10:04 UTC,NaN,NaN
3,4,horseradish,vegetable,odor,2011-10-02 06:10:05 UTC,2011-10-02 06:10:05 UTC,NaN,NaN
4,5,vegetable,vegetable,odor,2011-10-02 06:10:05 UTC,2011-10-02 06:10:05 UTC,NaN,NaN


In [9]:
flavor.shape

(883, 8)

In [10]:
flavor["flavor_group"].unique()  # 87% of the rows in this column are empty

array(['vegetable', 'herbaceous', 'fatty', 'spicy', 'floral', 'balsamic',
       'citrus', 'woody', 'meaty', 'green', 'wine_like', 'earthy',
       'fruity', 'chemical', 'nutty', 'camphoraceous', 'tobacco', 'minty',
       'pepper', 'medicinal', 'mossy', 'fishy', 'smoky', 'musky', 'soapy',
       'seedy', 'animal', nan], dtype=object)

In [11]:
flavor["category"].unique() # why even have this column??? no empty rows, all the same value

array(['odor'], dtype=object)

In [12]:
flavor.isna().sum()

id                0
name              0
flavor_group    770
category          0
created_at        0
updated_at        0
creator_id      882
updater_id      881
dtype: int64

----
## food.csv

In [19]:
food = pd.read_csv("data/foodb_2020_04_07_csv/Food.csv")
food.head()

,id,name,name_scientific,description,itis_id,wikipedia_id,picture_file_name,picture_content_type,picture_file_size,picture_updated_at,...,food_type,created_at,updated_at,creator_id,updater_id,export_to_afcdb,category,ncbi_taxonomy_id,export_to_foodb,public_id
0,1,Angelica,Angelica keiskei,Angelica is a genus of about 60 species of tal...,NaN,Angelica,1.jpg,image/jpeg,111325.0,2012-04-20 09:29:57 UTC,...,Type 1,2011-02-09 00:37:14 UTC,2019-05-14 18:04:13 UTC,NaN,2.0,False,specific,357850.0,True,FOOD00001
1,2,Savoy cabbage,Brassica oleracea var. sabauda,Savoy cabbage (Brassica oleracea convar. capit...,NaN,Savoy cabbage,2.jpg,image/jpeg,155178.0,2012-04-20 09:39:54 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-14 18:04:13 UTC,NaN,NaN,False,specific,1216010.0,True,FOOD00002
2,3,Silver linden,Tilia argentea,Tilia tomentosa (Silver Lime in the UK and Sil...,845789.0,Tilia tomentosa,3.jpg,image/jpeg,56367.0,2012-04-20 09:41:25 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-17 16:19:45 UTC,NaN,NaN,False,specific,NaN,True,FOOD00003
3,4,Kiwi,Actinidia chinensis,"The kiwifruit, often shortened to kiwi in many...",506775.0,Kiwifruit,4.jpg,image/jpeg,110661.0,2012-04-20 09:32:21 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-14 18:04:13 UTC,NaN,NaN,False,specific,3625.0,True,FOOD00004
4,5,Allium,Allium,Allium haematochiton is a species of wild onio...,42634.0,Allium haematochiton,5.jpg,image/jpeg,341911.0,2012-04-20 09:37:44 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-14 18:04:13 UTC,NaN,NaN,False,specific,4678.0,True,FOOD00005


In [20]:
food.shape

(992, 23)

In [21]:
food.isna().sum()

id                        0
name                      0
name_scientific         317
description              47
itis_id                 382
wikipedia_id            139
picture_file_name       115
picture_content_type    115
picture_file_size       115
picture_updated_at      144
legacy_id               372
food_group                3
food_subgroup             7
food_type                 0
created_at                0
updated_at                0
creator_id              739
updater_id              694
export_to_afcdb           0
category                 91
ncbi_taxonomy_id        373
export_to_foodb           0
public_id                 0
dtype: int64

In [22]:
# looking at foods that have a description
desc_food = food.dropna(subset=["description"])
desc_food.isna().sum()

id                        0
name                      0
name_scientific         283
description               0
itis_id                 344
wikipedia_id            100
picture_file_name       107
picture_content_type    107
picture_file_size       107
picture_updated_at      136
legacy_id               339
food_group                2
food_subgroup             2
food_type                 0
created_at                0
updated_at                0
creator_id              719
updater_id              681
export_to_afcdb           0
category                 85
ncbi_taxonomy_id        332
export_to_foodb           0
public_id                 0
dtype: int64

In [23]:
# their "advanced search" seems to be hitting this "Food.csv" table: https://foodb.ca/unearth/advanced/foods
tastes = ["sour", "salt", "salty", "sweet", "bitter", "fat", "acid", "acidic", "umami"]
exclude = ["Not a significant source of Calories from Fat"]

In [24]:
desc_food.shape

(945, 23)

In [25]:
desc_food.loc[desc_food["description"].str.contains("salt")].head()

,id,name,name_scientific,description,itis_id,wikipedia_id,picture_file_name,picture_content_type,picture_file_size,picture_updated_at,...,food_type,created_at,updated_at,creator_id,updater_id,export_to_afcdb,category,ncbi_taxonomy_id,export_to_foodb,public_id
10,11,Cashew nut,Anacardium occidentale,"The cashew, or caju, is a small tree native to...",28793.0,Cashew_nut,11.jpg,image/jpeg,670639.0,2012-04-20 09:29:53 UTC,...,Type 1,2011-02-09 00:37:15 UTC,2019-05-14 18:04:13 UTC,NaN,NaN,False,specific,171929.0,True,FOOD00011
29,30,Common cabbage,Brassica oleracea,Brassica oleracea is the species of plant that...,23062.0,Brassica_oleracea,30.jpg,image/jpeg,146148.0,2012-04-20 09:35:56 UTC,...,Type 1,2011-02-09 00:37:16 UTC,2019-05-14 18:04:14 UTC,NaN,2.0,False,specific,3712.0,True,FOOD00030
111,112,Spearmint,Mentha spicata,Spearmint or spear mint (Mentha spicata) is a ...,32272.0,Spearmint,112.jpg,image/jpeg,90563.0,2012-04-20 09:30:01 UTC,...,Type 1,2011-02-09 00:37:20 UTC,2019-05-14 18:04:16 UTC,NaN,NaN,False,specific,29719.0,True,FOOD00112
138,139,Pepper (Spice),Piper nigrum,Black pepper (Piper nigrum) is a flowering vin...,18310.0,Black pepper,139.jpg,image/jpeg,165046.0,2012-04-20 09:36:32 UTC,...,Type 1,2011-02-09 00:37:22 UTC,2019-05-14 18:04:17 UTC,NaN,2.0,False,specific,13216.0,True,FOOD00139
149,150,Guava,Psidium guajava,Guavas (singular guava) are plants in the Myrt...,27240.0,Guava,150.jpg,image/jpeg,62868.0,2012-04-20 09:43:21 UTC,...,Type 1,2011-02-09 00:37:23 UTC,2019-05-14 18:04:17 UTC,NaN,NaN,False,specific,120290.0,True,FOOD00150


In [26]:
desc_food.loc[desc_food["description"].str.contains("salty")].head()

,id,name,name_scientific,description,itis_id,wikipedia_id,picture_file_name,picture_content_type,picture_file_size,picture_updated_at,...,food_type,created_at,updated_at,creator_id,updater_id,export_to_afcdb,category,ncbi_taxonomy_id,export_to_foodb,public_id
693,716,Soy sauce,NaN,Soy sauce (also called soya sauce) is a condim...,NaN,Soy_sauce,716.jpg,image/jpeg,8472.0,2012-04-20 09:44:17 UTC,...,Type 2,2011-12-12 23:57:21 UTC,2019-05-14 18:05:30 UTC,2.0,2.0,False,specific,NaN,True,FOOD00694
694,717,Miso,NaN,Miso is a traditional Japanese seasoning produ...,NaN,Miso,717.jpg,image/jpeg,66987.0,2012-04-20 09:31:18 UTC,...,Type 2,2011-12-13 00:03:37 UTC,2020-03-12 21:00:16 UTC,2.0,2.0,False,specific,NaN,True,FOOD00695
721,747,Corn chip,NaN,A corn chip is a snack food made from cornmeal...,NaN,Corn_chip,747.jpg,image/jpeg,108546.0,2012-04-20 09:30:12 UTC,...,Type 2,2011-12-15 17:31:29 UTC,2020-03-16 16:32:15 UTC,2.0,2.0,False,specific,NaN,False,FOOD00722


In [27]:
desc_food["food_type"].value_counts(normalize=True)

food_type
Type 1     0.733333
Type 2     0.248677
Unknown    0.017989
Name: proportion, dtype: float64

In [28]:
salty = desc_food.loc[desc_food["description"].str.contains("salty")]
salty["food_type"].value_counts(normalize=True)

food_type
Type 2    1.0
Name: proportion, dtype: float64

In [29]:
salt = desc_food.loc[desc_food["description"].str.contains("salt")]
salt["food_type"].value_counts(normalize=True)

food_type
Type 1    0.5
Type 2    0.5
Name: proportion, dtype: float64

In [30]:
salt.shape

(44, 23)

In [31]:
salt["food_type"].unique()

array(['Type 1', 'Type 2'], dtype=object)

In [34]:
# what is food "Type 1"?
food_types = salt["food_type"].unique()
salt_0 = salt.loc[salt["food_type"] == food_types[0]]
print(salt_0["name"].unique())
print(len(salt_0["name"].unique()))
salt_0.shape

['Cashew nut' 'Common cabbage' 'Spearmint' 'Pepper (Spice)' 'Guava'
 'Striped bass' 'Bluefish' 'Natal plum' 'Nuttall cockle' 'King mackerel'
 'Common ling' 'Sea trout' 'Whitefish' 'Smelt'
 'Bivalvia (Clam, Mussel, Oyster)' 'Crayfish' 'Spot croaker' 'Anchovy'
 'Marine mussel' 'True sole' 'Salted butter' 'Green plum']
22


(22, 23)

In [35]:
# what is food "Type 2"?
salt_1 = salt.loc[salt["food_type"] == food_types[1]]
print(salt_1["name"].unique())
print(len(salt_1["name"].unique()))
salt_1.shape

['Madeira wine' 'Salad dressing' 'Sauce' 'Salt' 'Sausage' 'Mustard'
 'Soy sauce' 'Miso' 'Baked beans' 'Processed cheese' 'Potato chip'
 'Tortilla chip' 'Corn chip' 'Adobo' 'Hushpuppy' 'Mixed nuts' 'Cracker'
 'Focaccia' 'Bagel' 'French toast' 'Stuffing' 'Hummus']
22


(22, 23)

----
## content.csv

In [36]:
content = pd.read_csv("data/foodb_2020_04_07_csv/Content.csv")
content.head()

/var/folders/sm/pxzljrp530ddx9cxpl80jslh0000gn/T/ipykernel_33818/1004614589.py:1: DtypeWarning: Columns (4,5,6,7,8,9,13,14,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  content = pd.read_csv("data/foodb_2020_04_07_csv/Content.csv")


,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
0,1,1,Nutrient,4,29,Kiwi,Actinidia chinensis PLANCHON [Actinidiaceae],Fruit,FAT,FAT,...,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,1955.00,raw,0
1,2,1,Nutrient,6,53,Onion,Allium cepa L. [Liliaceae],Bulb,FAT,FAT,...,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,1853.95,raw,0
2,3,1,Nutrient,6,53,Onion,Allium cepa L. [Liliaceae],Leaf,FAT,FAT,...,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,4150.00,raw,0
3,4,1,Nutrient,9,55,Chives,Allium schoenoprasum L. [Liliaceae],Leaf,FAT,FAT,...,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,3900.00,raw,0
4,5,1,Nutrient,11,70,Cashew,Anacardium occidentale L. [Anacardiaceae],Fruit,FAT,FAT,...,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,2500.00,other,0


In [37]:
# this is the ginormous table
print(f"shape: {content.shape}")
print(f"columns: {content.columns}")

shape: (5145532, 26)
columns: Index(['id', 'source_id', 'source_type', 'food_id', 'orig_food_id',
       'orig_food_common_name', 'orig_food_scientific_name', 'orig_food_part',
       'orig_source_id', 'orig_source_name', 'orig_content', 'orig_min',
       'orig_max', 'orig_unit', 'orig_citation', 'citation', 'citation_type',
       'creator_id', 'updater_id', 'created_at', 'updated_at', 'orig_method',
       'orig_unit_expression', 'standard_content', 'preparation_type',
       'export'],
      dtype='object')


In [38]:
content.isna().sum()

id                                 0
source_id                          0
source_type                        0
food_id                            0
orig_food_id                 4308143
orig_food_common_name         971656
orig_food_scientific_name    5099435
orig_food_part               5104956
orig_source_id               4448551
orig_source_name             4445227
orig_content                 4289574
orig_min                     5099438
orig_max                     5099476
orig_unit                    4286563
orig_citation                5142217
citation                           0
citation_type                      0
creator_id                   5145532
updater_id                   5145532
created_at                         0
updated_at                         0
orig_method                  5138235
orig_unit_expression         5139940
standard_content             4286932
preparation_type             1324146
export                             0
dtype: int64

In [39]:
common_names = content["orig_food_common_name"].unique()
len(common_names)

9461

In [40]:
common_names[:20]

array(['Kiwi', 'Onion', 'Chives', 'Cashew', 'Pineapple', 'Dill',
       'Custard Apple', 'Celery', 'Groundnut', 'Burdock', 'Horseradish',
       'Tarragon', 'Mugwort', 'Asparagus', 'Oats', 'Carambola',
       'Brazilnut', 'Beebread', 'Mustard Greens', 'Brussel-Sprout'],
      dtype=object)

In [41]:
content["preparation_type"].unique()

array(['raw', 'other', 'other (raw)', 'tea or coffee', nan, 'cooked',
       'raw, frozen', 'dried or powder', 'other (raw, frozen)',
       'beverage (frozen concentrated)', 'oil', 'Oil', 'beverage (juice)',
       'beverage (frozen)', 'other (beverage)',
       'beverage (juice, concentrated)', 'puree',
       'other (dried or powder)', 'beverage (Alcoholic)', 'No', 'no',
       'dairy product', 'Dairy product', 'cooked or raw',
       'beverage (juice, added ascorbic acid)',
       'beverage (juice, added ascorbic acid and calcium)',
       'beverage (ascorbic acid added)', 'raw, frozed', 'blanching water',
       'raw, dried', 'raw, frozen, dried'], dtype=object)

In [ ]:
# content.loc[content["orig_food_part"] == "Hull Husk"]
# I feel like this would be more meaningful to me if I were a chemist

In [42]:
# according to that schema page, I think the "food_id" column here should map the the "id" column on the "food" table
print(f"unique food_ids in 'content' table: {len(content['food_id'].unique())}")
print(f"rows in 'food' table: {food.shape[0]}")

unique food_ids in 'content' table: 987
rows in 'food' table: 992


- can I reference the descriptions in the "food" table by the food_id in the content table?

In [43]:
set(content["food_id"].unique()).issubset(set(food["id"].unique()))

False

In [44]:
content_ids = set(content["food_id"].unique()).difference(set(food["id"].unique()))

In [45]:
set(food["id"].unique()).difference(set(content["food_id"].unique()))

{278, 512, 628, 794, 828, 932, 934, 966, 980, 993}

In [46]:
# out of all the foods in the content table, this is how many have ids that are not in the food table
#  not a big number out of 5,145,532
content.loc[content["food_id"].isin(list(content_ids))].shape

(2826, 26)

In [47]:
content["orig_food_part"].unique()

array(['Fruit', 'Bulb', 'Leaf', 'Seed', 'Plant', 'Root', 'Shoot', 'Stem',
       'Flower', 'Latex Exudate', 'Bark', 'Silk Stigma Style', 'Rhizome',
       'Stem Bark', 'Sprout Seedling', 'Pericarp', 'Hay', 'Aril', 'Twig',
       'Herb', 'Pt', 'Tuber', 'Bud', 'Hull Husk', 'Pith', nan, 'Seed Oil',
       'Essential Oil', 'Epidermis', 'Fruit Juice', 'Leaf Essent. Oil',
       'Gum', 'Resin, Exudate, Sap', 'Seed Essent. Oil',
       'Shoot Essent. Oil', 'Root Essent. Oil', 'Tissue Culture',
       'Stem Essent. Oil', 'Leaf Wax', 'Petiole', 'Oil', 'Bran', 'Embryo',
       'Inflorescence', 'Fruit Essent. Oil', 'Root Bark', 'Anther',
       'Seed Coat', 'Endosperm', 'Cotyledon', 'Pollen Or Spore',
       'Flower Essent. Oil', 'Bark Essent. Oil', 'Wood Essent. Oil',
       'Pericarp Essent. Oil', 'Exocarp', 'Branches', 'Wood',
       'Rhizome Essent. Oil', 'Calyx', 'Hs', 'Testa', 'Hypocotyl',
       'Nodule', 'Leaf Diffusate', 'Protoplast', 'Caryopsis',
       'Heart Wood', 'Fruit Epidermis', 

In [48]:
content["source_type"].value_counts()

source_type
Compound    5007500
Nutrient     138032
Name: count, dtype: int64

----
## tying the recipe to the food database

In [80]:
foods = []
for ingredients in df["ingredient_details"]:
    for item in ingredients:
        foods.append(item["food"])
        
print(set(foods))

{'oregano', 'rocket', 'egg', 'chilli', 'sourdough', 'pepper', 'black pepper', 'pesto', 'olive oil', 'sourdough bread', 'avocado', 'salt', 'water', 'mushroom', 'eggs', 'ham', 'sea salt', 'cherry tomatoes', 'cherry tomato', 'Salt', 'garlic', 'ricotta cheese', 'cheddar cheese', 'spinach', 'balsamic vinegar', 'lamb racks'}


In [68]:
# content.loc[content["orig_food_common_name"].str.lower() == "egg"].shape  # 0 results
content.loc[content["orig_food_common_name"].str.lower() == "eggs"].shape

(21, 26)

In [69]:
content.loc[content["orig_food_common_name"].str.lower() == "spinach"].shape

(192, 26)

In [73]:
# content.loc[content["orig_food_common_name"].str.lower() == "cherry tomatoes"].shape  # 0 results
# content.loc[content["orig_food_common_name"].str.lower() == "cherry tomato"].shape  # 0 results
# content.loc[content["orig_food_common_name"].str.lower() == "tomatoes"].shape  # 127 results
content.loc[content["orig_food_common_name"].str.lower() == "tomato"].shape  # 242 results

(242, 26)

In [77]:
# content.loc[content["orig_food_common_name"].str.lower() == "rocket"].shape  # 0 results
# content.loc[content["orig_food_common_name"].str.lower() == "arugula"].shape  # 0 results

# food.loc[food["name"].str.lower() == "arugula"].shape  # 0 results
food.loc[food["name"].str.lower() == "rocket"].shape  # 0 results

(0, 23)

In [78]:
content.loc[content["orig_food_common_name"].str.lower() == "olive oil"].shape

(212, 26)

In [81]:
content.loc[content["orig_food_common_name"].str.lower() == "oregano"].shape

(114, 26)

In [83]:
# content.loc[content["orig_food_common_name"].str.lower() == "chilli"].shape  # 0 results
content.loc[content["orig_food_common_name"].str.lower() == "chili"].shape

(0, 26)

In [86]:
# content.loc[content["orig_food_common_name"].str.lower() == "sourdough"].shape  # 0 results
# content.loc[content["orig_food_common_name"].str.lower() == "sourdough bread"].shape  # 0 results
content.loc[content["orig_food_common_name"].str.lower() == "bread"].shape

(0, 26)

In [64]:
content.loc[content["orig_food_common_name"] == "onion"].shape

(22, 26)

In [65]:
content.loc[content["orig_food_common_name"] == "Onion"].shape

(387, 26)

In [66]:
content.loc[content["orig_food_common_name"].str.lower() == "onion"].shape

(409, 26)